In [4]:
import os
os.chdir("x:\DL\Projects\MLOPs")
%pwd

'x:\\DL\\Projects\\MLOPs'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data: Path
    unzip_dir: Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file_dir: Path
    req_files: list
    
@dataclass(frozen=True)
class ModelTrainerConfig:
    model: str
    save_path: Path
    epochs: int
    batch: int
    imgsz: int

@dataclass(frozen=True)
class MlflowConfig:
    mlflow_uri: str
    experiment_name: str
    model_name: str

@dataclass(frozen=True)
class EvaluationConfig:
    all_params: dict
    base_model: Path

In [9]:
import os
from scripts.MLOPs.constants import *
from scripts.MLOPs.utils.common import read_yaml, create_directories
#from scripts.MLOPs.entity.config_entity import *

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_dataingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data= config.local_data,
            unzip_dir= config.unzip_dir
            )
        return data_ingestion_config
    
    def get_datavalidation_config(self)->DataValidationConfig:
        config = self.config.data_validation
        create_directories([config.data_val_dir])
        data_validation_config = DataValidationConfig(
            root_dir=config.data_val_dir,
            status_file_dir= config.data_val_status,
            req_files= config.data_val_req
            )
        return data_validation_config
    
    def get_model_trainer(self)-> ModelTrainerConfig:
        config = self.config.training_params
        modelparams = ModelTrainerConfig(
            model= config.model,
            save_path= config.save_path,
            epochs= config.epochs,
            batch= config.batch,
            imgsz= config.imgsz
        )
        return modelparams
    
    def get_mlflow_config(self)-> MlflowConfig:
        config = self.config.mlflow
        mlflow_config = MlflowConfig(
            mlflow_uri= config.mlflow_uri,
            experiment_name= config.experiment_name,
            model_name= config.model_name,
            params = self.params
        )
        return mlflow_config
    
    def get_all_params(self)-> EvaluationConfig:
        config = self.config.training_params
        eval_config = EvaluationConfig(
            base_model = os.path.join(config.save_path,"/Trainedv8.pt"),
            all_params = self.params
        )
        return eval_config
    
    

In [ ]:
from ultralytics import YOLO, settings
import os,sys
import logging
import torch
from urllib.parse import urlparse
import mlflow
from scripts.MLOPs.exception import AppException



class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config


    @staticmethod
    def load_model(path:path):
        return YOLO(path)
    
    


In [10]:

def find_indices(large_list, small_subset):
    indices = []
    for item in small_subset:
        item_indices = [i for i, x in enumerate(large_list) if x == item]
        indices.extend(item_indices)
    return indices


large_list = ['a', 'b', 'c', 'a', 'd', 'e', 'a', 'f']
small_subset = ['a', 'e']

indices = find_indices(large_list, small_subset)
print(indices)  # Output: [0, 3, 6, 5]

[0, 3, 6, 5]
